In [62]:

# %pip install numpy
# %pip install torch
# %pip install tqdm
# %pip install transformers
# %pip install tensorboard
# %pip install scikit-learn
# %pip install PyYAML
# %pip install click
# %pip install sentencepiece
# %pip install mlflow


In [20]:
import json
import logging
import os
import sys

import torch
import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers.trainer_utils import is_main_process
sys.path.append("..")

from contract_nli.conf import load_conf
from contract_nli.dataset.dataset import load_and_cache_examples, load_and_cache_features
from contract_nli.dataset.encoder import SPAN_TOKEN
from contract_nli.evaluation import evaluate_all
from contract_nli.model.identification_classification import MODEL_TYPE_TO_CLASS, update_config
from contract_nli.model.classification import BertForClassification
from contract_nli.postprocess import format_json
from contract_nli.predictor import predict, predict_classification
from contract_nli.trainer import Trainer, setup_optimizer
from contract_nli.utils import set_seed, distributed_barrier

# import mlflow

logger = logging.getLogger(__name__)

In [21]:
logger = logging.getLogger(__name__)

In [82]:
dev_dataset_path = "../data/converted/dev_data_53.json"
model_dir = "../models/"
dataset_path = "../data/converted/test_data_107.json"
output_prefix = "../reports/"

In [83]:
import yaml
def load_conf(path):
    with open(path) as fin:
        conf_txt = fin.read()
    conf = yaml.load(conf_txt, Loader=yaml.SafeLoader)
    return conf

In [84]:
conf: dict = load_conf(os.path.join(model_dir, 'conf.yml'))

device = torch.device("cuda" if torch.cuda.is_available() and not conf['no_cuda'] else "cpu")
print(device)
print(conf['no_cuda'])
# conf['no_cuda'] = True
n_gpu = 0 if conf['no_cuda'] else torch.cuda.device_count()

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

cpu
False


In [85]:
conf['cache_dir'] = '.'

In [86]:
transformers.utils.logging.set_verbosity_info()
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

logger.info("Loading models with following conf %s",
            {k: v for k, v in conf.items() if k != 'raw_yaml'})

tokenizer = AutoTokenizer.from_pretrained(
    model_dir,
    do_lower_case=conf['do_lower_case'],
    cache_dir=conf['cache_dir'],
    use_fast=False
)
config = AutoConfig.from_pretrained(
    model_dir,
    cache_dir=conf['cache_dir']
)

03/14/2023 01:05:44 - INFO - __main__ -   Loading models with following conf {'model_name_or_path': 'bert-base-uncased', 'train_file': '/dbfs/FileStore/span_nli_bert/data/spannli_train_data.json', 'dev_file': '/dbfs/FileStore/span_nli_bert/data/spannli_dev_data.json', 'config_name': None, 'tokenizer_name': None, 'cache_dir': '.', 'max_seq_length': 512, 'doc_stride': 64, 'max_query_length': 256, 'do_lower_case': True, 'per_gpu_train_batch_size': 8, 'per_gpu_eval_batch_size': 8, 'learning_rate': 3e-05, 'gradient_accumulation_steps': 1, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_epochs': 5.0, 'max_steps': None, 'warmup_steps': 200, 'lang_id': None, 'valid_steps': 3000, 'early_stopping': True, 'save_steps': -1, 'seed': 42, 'fp16': False, 'fp16_opt_level': 'O1', 'no_cuda': False, 'overwrite_cache': False, 'weight_class_probs_by_span_probs': True, 'class_loss_weight': 0.1, 'task': 'identification_classification', 'symbol_based_hypothesis': False}
[INFO|tokenizatio

In [87]:
calibration_coeff = None
if dev_dataset_path is not None:
    if conf['task'] != 'identification_classification':
        raise click.BadOptionUsage(
            '--dev-dataset-path',
            '--dev-dataset-path cannot be used when the task is not identification_classification')
    print("Cache Examples!!!")
    examples = load_and_cache_examples(
        dev_dataset_path,
        local_rank=-1,
        overwrite_cache=True,
        cache_dir=conf['cache_dir']
    )
    print("Create features!!!")
    dataset, features = load_and_cache_features(
        dev_dataset_path,
        examples,
        tokenizer,
        max_seq_length=conf['max_seq_length'],
        doc_stride=conf.get('doc_stride', None),
        max_query_length=conf['max_query_length'],
        dataset_type=conf['task'],
        symbol_based_hypothesis=conf['symbol_based_hypothesis'],
        threads=None,
        local_rank=-1,
        overwrite_cache=conf['overwrite_cache'],
        labels_available=True,
        cache_dir=conf['cache_dir']
    )
    

03/14/2023 01:05:46 - INFO - contract_nli.dataset.dataset -   Creating examples from dataset file at ../data/converted/dev_data_53.json


Cache Examples!!!


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:04<00:00, 12.46it/s]
03/14/2023 01:05:50 - INFO - contract_nli.dataset.dataset -   Saving examples into cached file ./cached_examples_dev_data_53
03/14/2023 01:05:54 - INFO - contract_nli.dataset.dataset -   Loading features from cached file ./cached_features_dev_data_53_identification_classification__512_256_64


Create features!!!


In [28]:
print(len(examples), len(features), len(dataset))

867 4636 4636


In [29]:
print(type(examples), type(features), type(dataset))

<class 'list'> <class 'list'> <class 'torch.utils.data.dataset.TensorDataset'>


In [30]:
if conf['task'] == 'identification_classification':
    model = MODEL_TYPE_TO_CLASS[config.model_type].from_pretrained(
        model_dir, cache_dir=conf['cache_dir']
    )
else:
    model = BertForClassification.from_pretrained(
        model_dir, cache_dir=conf['cache_dir'])

model.to(device)
print("Hello")
if dev_dataset_path is not None:
    all_results = predict(
        model, dataset, examples, features,
        per_gpu_batch_size=conf['per_gpu_eval_batch_size'],
        device=device, n_gpu=n_gpu,
        weight_class_probs_by_span_probs=conf[
            'weight_class_probs_by_span_probs'])
    calibration_coeff = compute_prob_calibration_coeff(
        examples, all_results)

[INFO|configuration_utils.py:658] 2023-03-13 21:47:41,646 >> loading configuration file ../models/config.json
[INFO|configuration_utils.py:712] 2023-03-13 21:47:41,647 >> Model config BertConfig {
  "_name_or_path": "/dbfs/FileStore/span_nli_bert/model/span_BERT_FINETUNED_FULL_5/best-checkpoint",
  "architectures": [
    "BertForIdentificationClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "class_loss_weight": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "impossible_strategy": "ignore",
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30523

Hello


Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 580/580 [33:23<00:00,  3.45s/it]


0 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026faf0>
1 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026f610>
2 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026f8b0>
3 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026ffa0>
4 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026f5b0>
5 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026f970>
6 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026fb20>
7 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026f640>
8 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026fca0>
9 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026fbb0>
10 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026fd60>
11 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026ff70>
12 <contract_nli.dataset.loader.ContractNLIExample object at 0x22026fa90>
13 <contract_nli.dataset.loader.ContractNLIExamp

116 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028c0a0>
117 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028c250>
118 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028c3a0>
119 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028c580>
120 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028cfd0>
121 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028c8e0>
122 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028c910>
123 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028ce20>
124 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028ce50>
125 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028c370>
126 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028cd90>
127 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028cd60>
128 <contract_nli.dataset.loader.ContractNLIExample object at 0x22028ca60>
129 <contract_nli.dataset

247 <contract_nli.dataset.loader.ContractNLIExample object at 0x225b82be0>
248 <contract_nli.dataset.loader.ContractNLIExample object at 0x225b82b80>
249 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f67040>
250 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f67070>
251 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f670a0>
252 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f670d0>
253 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f67100>
254 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f67130>
255 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f67250>
256 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f67160>
257 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f67190>
258 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f673a0>
259 <contract_nli.dataset.loader.ContractNLIExample object at 0x229f673d0>
260 <contract_nli.dataset

513 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d49370>
514 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d49070>
515 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d490a0>
516 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d490d0>
517 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d49040>
518 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d493a0>
519 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d49100>
520 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d493d0>
521 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d49400>
522 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d49460>
523 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d49430>
524 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d49490>
525 <contract_nli.dataset.loader.ContractNLIExample object at 0x232d494c0>
526 <contract_nli.dataset

650 <contract_nli.dataset.loader.ContractNLIExample object at 0x23491bc70>
651 <contract_nli.dataset.loader.ContractNLIExample object at 0x23491bd00>
652 <contract_nli.dataset.loader.ContractNLIExample object at 0x23491bcd0>
653 <contract_nli.dataset.loader.ContractNLIExample object at 0x23491beb0>
654 <contract_nli.dataset.loader.ContractNLIExample object at 0x23491bee0>
655 <contract_nli.dataset.loader.ContractNLIExample object at 0x23491bf10>
656 <contract_nli.dataset.loader.ContractNLIExample object at 0x23491bf40>
657 <contract_nli.dataset.loader.ContractNLIExample object at 0x23491bf70>
658 <contract_nli.dataset.loader.ContractNLIExample object at 0x23491bfa0>
659 <contract_nli.dataset.loader.ContractNLIExample object at 0x23491bfd0>
660 <contract_nli.dataset.loader.ContractNLIExample object at 0x236df2040>
661 <contract_nli.dataset.loader.ContractNLIExample object at 0x236df2070>
662 <contract_nli.dataset.loader.ContractNLIExample object at 0x236df20a0>
663 <contract_nli.dataset

822 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866ea90>
823 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866eaf0>
824 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866e970>
825 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866ea00>
826 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866eac0>
827 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866e9a0>
828 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866ee50>
829 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866eeb0>
830 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866ee80>
831 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866eee0>
832 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866ef10>
833 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866efa0>
834 <contract_nli.dataset.loader.ContractNLIExample object at 0x23866ef40>
835 <contract_nli.dataset

In [33]:
from contract_nli.postprocess import format_json, compute_prob_calibration_coeff

calibration_coeff = compute_prob_calibration_coeff(
        examples, all_results)

In [34]:
calibration_coeff

-0.4375740720577821

In [108]:
examples = load_and_cache_examples(
    dataset_path,
    local_rank=-1,
    overwrite_cache=True,
    cache_dir=conf['cache_dir']
)

03/14/2023 01:17:41 - INFO - contract_nli.dataset.dataset -   Creating examples from dataset file at ../data/converted/test_data_107.json
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 14.88it/s]
03/14/2023 01:17:43 - INFO - contract_nli.dataset.dataset -   Saving examples into cached file ./cached_examples_test_data_107


In [109]:

dataset, features = load_and_cache_features(
    dataset_path,
    examples,
    tokenizer,
    max_seq_length=conf['max_seq_length'],
    doc_stride=conf.get('doc_stride', None),
    max_query_length=conf['max_query_length'],
    dataset_type=conf['task'],
    symbol_based_hypothesis=conf['symbol_based_hypothesis'],
    threads=None,
    local_rank=-1,
    overwrite_cache=conf['overwrite_cache'],
    labels_available=True,
    cache_dir=conf['cache_dir']
)

03/14/2023 01:17:44 - INFO - contract_nli.dataset.dataset -   Loading features from cached file ./cached_features_test_data_107_identification_classification__512_256_64


In [110]:
if conf['task'] == 'identification_classification':
    all_results = predict(
        model, dataset, examples, features,
        per_gpu_batch_size=conf['per_gpu_eval_batch_size'],
        device=device, n_gpu=n_gpu,
        weight_class_probs_by_span_probs=conf['weight_class_probs_by_span_probs'],
        calibration_coeff=calibration_coeff)
else:
    all_results = predict_classification(
        model, dataset, features,
        per_gpu_batch_size=conf['per_gpu_eval_batch_size'],
        device=device, n_gpu=n_gpu)

03/14/2023 01:17:45 - INFO - contract_nli.predictor -   ***** Running evaluation *****
03/14/2023 01:17:45 - INFO - contract_nli.predictor -     Num examples = 1943
03/14/2023 01:17:45 - INFO - contract_nli.predictor -     Batch size = 8
Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 243/243 [12:23<00:00,  3.06s/it]


0 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b9492580>
1 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b9492160>
2 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b94926a0>
3 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b9492400>
4 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b9492ac0>
5 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b9492100>
6 <contract_nli.dataset.loader.ContractNLIExample object at 0x1bc12d5b0>
7 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b9492340>
8 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b94923a0>
9 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b94923d0>
10 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b9492070>
11 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b94926d0>
12 <contract_nli.dataset.loader.ContractNLIExample object at 0x1b9492490>
13 <contract_nli.dataset.loader.ContractNLIExamp

121 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43dd30>
122 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43dd60>
123 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43dd90>
124 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43ddc0>
125 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43ddf0>
126 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43de20>
127 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43de50>
128 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43de80>
129 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43deb0>
130 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43dee0>
131 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43df10>
132 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43df40>
133 <contract_nli.dataset.loader.ContractNLIExample object at 0x19c43df70>
134 <contract_nli.dataset

233 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca2b0>
234 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca2e0>
235 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca310>
236 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca340>
237 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca370>
238 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca460>
239 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca3a0>
240 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca3d0>
241 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca400>
242 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca490>
243 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca640>
244 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca670>
245 <contract_nli.dataset.loader.ContractNLIExample object at 0x1db6ca6a0>
246 <contract_nli.dataset

In [112]:
result_json = format_json(examples, all_results)
with open(output_prefix + 'full_result.json', 'w') as fout:
    json.dump(result_json, fout, indent=2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [148]:
dataset_path

'../data/converted/test_data_107.json'

In [149]:
# Copyright (c) 2021, Hitachi America Ltd. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from collections import defaultdict
from typing import Dict, List, Union

import numpy as np
import sklearn.metrics
from scipy.stats import hmean

from contract_nli.dataset.loader import NLILabel, ContractNLIExample
from contract_nli.postprocess import IdentificationClassificationResult, ClassificationResult


def evaluate_predicted_spans(y_true, y_pred) -> Dict[str, float]:
    if y_true.sum() == 0:
        # do not use zero_division=np.nan because it cannot distinguish
        # zero divisions from y_true and y_pred in f1_score
        recall = np.nan
        f1 = np.nan
    else:
        recall = sklearn.metrics.recall_score(y_true, y_pred)
        f1 = sklearn.metrics.f1_score(y_true, y_pred, zero_division=0)

    return {
        'precision': sklearn.metrics.precision_score(y_true, y_pred, zero_division=0),
        'recall': recall,
        'f1': f1,
        'accuracy': sklearn.metrics.accuracy_score(y_true, y_pred),
    }


def precision_at_recall(y_true, y_prob, recall: float):
    assert 0. <= recall <= 1.0
    if len(y_true) == 0 or np.sum(y_true) == 0:
        return np.nan
    threshs = np.sort(np.unique(y_prob))[::-1]
    # (len(np.unique(y_prob)), len(y_prob)) where first axis show prediction at different thresh
    y_preds = y_prob[None, :] >= threshs[:, None]
    recalls = np.logical_and(y_true[None, :], y_preds).sum(axis=1) / np.sum(y_true)
    # check that recalls are monotonically increasing
    assert np.all(recalls == np.sort(recalls))
    # because of >= relationship, there exist at least one thresh that gives
    # recall score of 1.0
    thresh = threshs[np.where(recalls >= recall)[0][0]]
    y_pred = y_prob >= thresh
    return sklearn.metrics.precision_score(y_true, y_pred, zero_division=0.)


def evaluate_spans(y_true, y_prob) -> Dict[str, float]:
    assert y_prob.ndim == 1
    assert y_true.ndim == 1
    assert len(y_true) == len(y_prob)
    metrics = evaluate_predicted_spans(y_true, y_prob > 0.5)
    metrics.update({
        'roc_auc': sklearn.metrics.roc_auc_score(y_true, y_prob),
        'map': sklearn.metrics.average_precision_score(y_true, y_prob),
        'precision@recall80': precision_at_recall(y_true, y_prob, 0.8),
        'precision@recall90': precision_at_recall(y_true, y_prob, 0.9)
    })
    return metrics


def predict_at_k(y_prob, k):
    y_pred = np.zeros_like(y_prob)
    for j in np.argsort(y_prob)[::-1][:k]:
        y_pred[j] = 1
    assert y_pred.sum() == min(k, len(y_pred))
    return y_pred


def evaluate_class(y_true, y_prob) -> Dict[str, float]:
    assert y_prob.ndim == 2 and y_prob.shape[1] == 3
    assert y_true.ndim == 1
    assert len(y_true) == len(y_prob)
    y_pred = np.argmax(y_prob, axis=1)
    metrics = {
        'accuracy': sklearn.metrics.accuracy_score(y_true, y_pred)
    }
    for label in (NLILabel.YES, NLILabel.NO):
        ln = label.name.lower()
        _y_true = y_true == label.value
        _y_pred = y_pred == label.value
        if _y_true.sum() == 0:
            # do not use zero_division=np.nan because it cannot distinguish
            # zero divisions from y_true and y_pred in f1_score
            recall = np.nan
            f1 = np.nan
        else:
            recall = sklearn.metrics.recall_score(_y_true, _y_pred)
            f1 = sklearn.metrics.f1_score(_y_true, _y_pred, zero_division=0)
        metrics.update({
            f'precision_{ln}': sklearn.metrics.precision_score(_y_true, _y_pred, zero_division=0),
            f'recall_{ln}': recall,
            f'f1_{ln}': f1,
        })
    for m in ('precision', 'recall', 'f1'):
        m_e = metrics[f'{m}_{NLILabel.YES.name.lower()}']
        m_c = metrics[f'{m}_{NLILabel.NO.name.lower()}']
        if np.isnan(m_e) or np.isnan(m_c):
            metrics[f'{m}_mean'] = np.nan
            metrics[f'{m}_hmean'] = np.nan
        else:
            metrics[f'{m}_mean'] = np.mean((m_e, m_c))
            metrics[f'{m}_hmean'] = hmean((m_e, m_c))

    return metrics


def _macro_average(dicts: List[Dict[str, float]]):
    ret = dict()
    for k in dicts[0].keys():
        vals = [d[k] for d in dicts if not np.isnan(d[k])]
        ret[k] = sum(vals) / float(len(vals))
    return ret


def remove_NOT_KNOWN(y_pred):
    assert y_pred.shape[1] == 3
    y_bin = y_pred[:, [NLILabel.NO.value, NLILabel.YES.value]]
    y_bin = np.where(np.tile(np.sum(y_bin, axis=1, keepdims=True), [1, 2]) == 0,
                     0.5,
                     y_bin / np.sum(y_bin, axis=1, keepdims=True))
    y_pred = np.zeros((len(y_pred), 3), dtype=y_pred.dtype)
    y_pred[:, [NLILabel.NO.value, NLILabel.YES.value]] = y_bin
    return y_pred


def evaluate_all(
        dataset: dict,
        results: List[dict],
        ks: List[int],
        task: str
        ) -> dict:
    assert task in ['identification_classification', 'classification', 'identification']
    id_to_result = {r['id']: r for r in results}
    label_ids = sorted(results[0]['annotation_sets'][0]['annotations'].keys())
    class_names = [NLILabel(i).to_anno_name() for i in range(len(NLILabel))]
    assert label_ids == sorted(dataset['labels'].keys()) or task == 'classification'
    if task in ['identification_classification', 'identification']:
        span_probs = defaultdict(list)
        span_labels = defaultdict(list)
    if task in ['identification_classification', 'classification']:
        class_probs = defaultdict(list)
        class_labels = defaultdict(list)
    for document in dataset['documents']:
        result = id_to_result[document['id']]['annotation_sets'][0]['annotations']
        annotations = document['annotation_sets'][0]['annotations']
        for label_id in label_ids:
            if task == 'classification' and label_id not in result:
                continue
            if task in ['identification_classification', 'classification']:
                class_labels[label_id].append(NLILabel.from_str(annotations[label_id]['choice']).value)
                class_probs[label_id].append(
                    np.array([result[label_id]['class_probs'][n] for n in class_names]))
            if task in ['identification_classification', 'identification']:
                # FIXME: this calculates precision optimistically
                if NLILabel.from_str(annotations[label_id]['choice']) != NLILabel.NOT_KNOWN:
                    span_label = np.zeros(len(document['spans']))
                    for s in annotations[label_id]['spans']:
                        span_label[s] = 1
                    span_labels[label_id].append(span_label)
                    span_probs[label_id].append(np.array(result[label_id]['span_probs']))
    if task in ['identification_classification', 'classification']:
        binary_label_ids = [
            l for l in label_ids
            if NLILabel.NO.value in class_labels[l] and
               NLILabel.YES.value in class_labels[l]]
        # this is not necessarily true with some training dataset
        # but we have to assume this for our evaluation to be a fair comparison
        if not set(class_probs.keys()).issuperset(set(binary_label_ids)):
            raise ValueError(
                'Some label ids are not in prediction when they are valid label '
                f'ids. Pred: {class_probs.keys()}, Dataset: {binary_label_ids}')
    if task in ['identification_classification', 'identification']:
        preds_at_ks = {
            k: {label_id: [predict_at_k(y_prob, k) for y_prob in y_probs]
                for label_id, y_probs in span_probs.items()}
            for k in ks
        }

    metrics = dict()

    metrics['micro_label_micro_doc'] = dict()
    if task in ['identification_classification', 'classification']:
        metrics['micro_label_micro_doc']['class_binary'] = evaluate_class(
            np.concatenate([np.array(class_labels[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value]
                            for l in binary_label_ids]),
            remove_NOT_KNOWN(
                np.vstack([np.stack(class_probs[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value, :]
                           for l in binary_label_ids]))
        )
    if task == 'identification_classification':
        metrics['micro_label_micro_doc']['class'] = evaluate_class(
            np.concatenate([class_labels[l] for l in label_ids]),
            np.vstack([np.stack(class_probs[l]) for l in label_ids])
        )
    if task in ['identification_classification', 'identification']:
        y_true = np.concatenate([l for k in label_ids for l in span_labels[k]])
        metrics['micro_label_micro_doc']['span'] = evaluate_spans(
            y_true,
            np.concatenate([l for l in label_ids for l in span_probs[l]])
        )
        for k in ks:
            if 'nda-9' in label_ids:
                label_ids.remove('nda-9')
            y_pred = np.concatenate([p for l in label_ids for p in preds_at_ks[k][l]])
            metrics['micro_label_micro_doc']['span'].update({
                f'{n}@{k}': v for n, v in evaluate_predicted_spans(y_true, y_pred).items()
            })
    metrics['macro_label_micro_doc'] = dict()
    if task in ['identification_classification', 'classification']:
        metrics['macro_label_micro_doc']['class_binary'] = _macro_average([
            evaluate_class(
                np.array(class_labels[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value],
                remove_NOT_KNOWN(np.stack(class_probs[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value, :]))
            for l in binary_label_ids
        ])
    if task == 'identification_classification':
        metrics['macro_label_micro_doc']['class'] = _macro_average([
            evaluate_class(np.array(class_labels[l]), np.stack(class_probs[l]))
            for l in label_ids
        ])
    if task in ['identification_classification', 'identification']:
        metrics['macro_label_micro_doc']['span'] = _macro_average([
            {
                **evaluate_spans(
                    np.concatenate(span_labels[l]),
                    np.concatenate(span_probs[l])),
                **{
                    f'{n}@{k}': v
                    for k in ks
                    for n, v in evaluate_predicted_spans(
                        np.concatenate(span_labels[l]),
                        np.concatenate(preds_at_ks[k][l])).items()
                }
            }
            for l in label_ids
        ])

    if task in ['identification_classification', 'identification']:
        metrics['macro_label_macro_doc'] = dict()
        metrics['macro_label_macro_doc']['span'] = _macro_average([
            _macro_average([
                {
                   **evaluate_spans(span_labels[l][i], span_probs[l][i]),
                   **{
                       f'{n}@{k}': v
                       for k in ks
                       for n, v in evaluate_predicted_spans(
                           span_labels[l][i],
                           preds_at_ks[k][l][i]).items()
                   }
                }
                for i in range(len(span_labels[l]))
            ])
            for l in label_ids
        ])

    metrics['micro_label_macro_doc'] = dict()
    if task == 'identification_classification':
        metrics['micro_label_macro_doc']['class'] = _macro_average([
            evaluate_class(
                np.array([class_labels[l][i] for l in label_ids]),
                np.stack([class_probs[l][i] for l in label_ids]))
            for i in range(len(class_labels[label_ids[0]]))
        ])
    if task in ['identification_classification', 'identification']:
        metrics['micro_label_macro_doc']['span'] = _macro_average([
            evaluate_spans(_l, _p)
            for l in label_ids
            for _l, _p in zip(span_labels[l], span_probs[l])
        ])
        metrics['micro_label_macro_doc']['span'].update({
            key: value
            for k in ks
            for key, value in _macro_average([
                {
                    f'{n}@{k}': v
                    for n, v in evaluate_predicted_spans(_l, _p).items()
                }
                for l in label_ids
                for _l, _p in zip(span_labels[l], preds_at_ks[k][l])
            ]).items()
        })

    metrics['label_wise'] = dict()
    for l in label_ids:
        metrics['label_wise'][l] = dict()
        metrics['label_wise'][l]['micro_doc'] = dict()
        if task in ['identification_classification', 'classification']:
            metrics['label_wise'][l]['micro_doc']['class_binary'] = evaluate_class(
                np.array(class_labels[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value],
                remove_NOT_KNOWN(np.stack(class_probs[l])[np.array(class_labels[l]) != NLILabel.NOT_KNOWN.value, :]))
            if not (NLILabel.NO.value in class_labels[l] and NLILabel.YES.value in class_labels[l]):
                metrics['label_wise'][l]['micro_doc']['class_binary'] = {
                    k: np.nan for k in metrics['label_wise'][l]['micro_doc']['class_binary'].keys()
                }
        if task == 'identification_classification':
            metrics['label_wise'][l]['micro_doc']['class'] = evaluate_class(
                np.array(class_labels[l]), np.stack(class_probs[l]))
        if task in ['identification_classification', 'identification']:
            y_true = np.concatenate(span_labels[l])
            metrics['label_wise'][l]['micro_doc']['span'] = {
                **evaluate_spans(y_true, np.concatenate(span_probs[l])),
                **{
                    f'{n}@{k}': v
                    for k in ks
                    for n, v in evaluate_predicted_spans(
                        y_true, np.concatenate(preds_at_ks[k][l])).items()
                }
            }
            metrics['label_wise'][l]['macro_doc'] = dict()
            metrics['label_wise'][l]['macro_doc']['span'] = _macro_average([
                {
                    **evaluate_spans(span_labels[l][i], span_probs[l][i]),
                    **{
                        f'{n}@{k}': v
                        for k in ks
                        for n, v in evaluate_predicted_spans(
                            span_labels[l][i], preds_at_ks[k][l][i]).items()
                    }
                }
                for i in range(len(span_labels[l]))
            ])
    return metrics

In [151]:
import json
# from contract_nli.evaluation import evaluate_all


result_json = json.load(open(output_prefix + 'full_result.json'))
len(result_json)

21

In [152]:

with open(dataset_path) as fin:
    test_dataset = json.load(fin)
    
metrics = evaluate_all(test_dataset, result_json,
                       [1, 3, 5, 8, 10, 15, 20, 30, 40, 50],
                       "classification")
logger.info(f"Results@: {json.dumps(metrics, indent=2)}")
with open(output_prefix + 'class_full_metrics.json', 'w') as fout:
    json.dump(metrics, fout, indent=2)

03/14/2023 01:51:27 - INFO - __main__ -   Results@: {
  "micro_label_micro_doc": {
    "class_binary": {
      "accuracy": 0.9090909090909091,
      "precision_yes": 0.9411764705882353,
      "recall_yes": 0.8727272727272727,
      "f1_yes": 0.9056603773584905,
      "precision_no": 0.8813559322033898,
      "recall_no": 0.9454545454545454,
      "f1_no": 0.9122807017543859,
      "precision_mean": 0.9112662013958126,
      "precision_hmean": 0.9102844638949672,
      "recall_mean": 0.9090909090909091,
      "recall_hmean": 0.9076363636363635,
      "f1_mean": 0.9089705395564383,
      "f1_hmean": 0.9089584850691915
    }
  },
  "macro_label_micro_doc": {
    "class_binary": {
      "accuracy": 0.8941879940235203,
      "precision_yes": 0.5869047619047619,
      "recall_yes": 0.6,
      "f1_yes": 0.5908075002902589,
      "precision_no": 0.6728013945119209,
      "recall_no": 0.7083333333333333,
      "f1_no": 0.6817931902490726,
      "precision_mean": 0.6298530782083414,
      "preci

In [162]:
len(result_json)

21

In [163]:
len(test_dataset['documents'])

21

In [154]:
import json
result_json = json.load(open(output_prefix + 'full_result.json'))

In [155]:

with open(dataset_path) as finn:
    test_dataset = json.load(finn)
    
metrics = evaluate_all(test_dataset, result_json,
                       [1, 3, 5, 8, 10, 15, 20, 30, 40, 50],
                       'identification')
# logger.info(f"Results@: {json.dumps(metrics, indent=2)}")
with open(output_prefix + 'span_full_metrics.json', 'w') as fout:
    json.dump(metrics, fout, indent=2)

ValueError: Found input variables with inconsistent numbers of samples: [5047, 4852]

In [0]:
with open(output_prefix + 'metrics.json', 'w') as fout:
    json.dump(metrics, fout, indent=2)